# Capstone 3: Choosing & Applying Clustering Algorithms

**Goal**: Compare clustering algorithms (KMeans, Agglomerative, DBSCAN), select champion model, and interpret clusters.

**Deliverables**:
- Algorithm comparison table (metrics: silhouette, DB index, runtime)
- Elbow plots (silhouette vs k, DB vs k)
- Cluster profiles and visualizations
- Champion model logged in `DECISIONS_LOG.md`

---
## A) Algorithm Comparison (Pros/Cons)

| Algorithm | Pros | Cons | Best For |
|-----------|------|------|----------|
| **K-Means** | Fast (O(nki)), interpretable centroids, well-validated in literature | Assumes spherical clusters, sensitive to outliers, requires pre-specifying k | Baseline; works when clusters are globular |
| **Agglomerative** | No need to pre-specify k, reveals hierarchy, deterministic | Slower (O(n² log n)), memory-intensive, less interpretable than centroids | Validate K-Means; explore sub-clusters |
| **DBSCAN** | Finds arbitrary shapes, handles noise/outliers, auto-detects k | Sensitive to eps/min_samples, struggles with varying density | Non-spherical patterns (e.g., linear commuter routes) |

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

# Import preprocessing modules
from src.paths import get_processed_file
from src.preprocess import prepare_clustering_features, create_preprocessing_pipeline

# Import clustering modules
from src.clustering import (
    run_kmeans,
    run_agglomerative,
    run_dbscan,
    compute_metrics,
    kmeans_elbow_analysis,
    stability_check
)

# Import interpretation modules
from src.interpretation import (
    describe_clusters,
    plot_cluster_profiles,
    plot_cluster_distributions,
    plot_hourly_weekday_heatmap,
    interpret_clusters,
    plot_cluster_comparison_table
)

print("✓ Libraries imported successfully")

---
## B) Load Cleaned Data & Prepare Features

In [ ]:
# Load cleaned dataset from Capstone 2
print("Loading cleaned dataset...")
df_clean = pd.read_csv(get_processed_file('trips_clean.csv'))

# Parse datetimes
df_clean['started_at'] = pd.to_datetime(df_clean['started_at'])
df_clean['ended_at'] = pd.to_datetime(df_clean['ended_at'])

print(f"✓ Loaded {len(df_clean):,} trips")
print(f"  Columns: {list(df_clean.columns)}")
print(f"\nFirst 3 rows:")
df_clean.head(3)

In [ ]:
# Prepare clustering features
X = prepare_clustering_features(df_clean)
print(f"Feature matrix shape: {X.shape}")
print(f"Features: {list(X.columns)}")

# Scale features
X_scaled, pipeline = create_preprocessing_pipeline(X, apply_pca=False, verbose=True)

print(f"\nScaled features ready for clustering: {X_scaled.shape}")

---
## C) Experiment 1: K-Means Elbow Analysis

Find optimal k by testing k ∈ {3, 4, 5, 6, 7}.

In [ ]:
# Elbow analysis
elbow_results = kmeans_elbow_analysis(
    X_scaled,
    k_range=[3, 4, 5, 6, 7],
    random_state=42,
    save=True
)

print("\nElbow Analysis Results:")
print(elbow_results)

In [ ]:
# Identify best k based on metrics
best_k_silhouette = elbow_results.loc[elbow_results['silhouette'].idxmax(), 'k']
best_k_db = elbow_results.loc[elbow_results['davies_bouldin'].idxmin(), 'k']

print(f"Best k by Silhouette: {best_k_silhouette} (score={elbow_results.loc[elbow_results['k']==best_k_silhouette, 'silhouette'].values[0]:.4f})")
print(f"Best k by DB Index: {best_k_db} (score={elbow_results.loc[elbow_results['k']==best_k_db, 'davies_bouldin'].values[0]:.4f})")

# Select k (prioritize silhouette, but check DB)
selected_k = int(best_k_silhouette)
print(f"\n→ Selected k = {selected_k} for further experiments")

---
## D) Experiment 2: Compare 3 Algorithms

Run K-Means, Agglomerative, and DBSCAN with selected k (or auto for DBSCAN).

In [ ]:
# Store results
results = {}

# 1. K-Means with selected k
print("\n" + "="*60)
print("EXPERIMENT: K-MEANS")
print("="*60)
start = time.time()
labels_km, model_km = run_kmeans(X_scaled, k=selected_k, n_init=20, random_state=42)
runtime_km = time.time() - start
metrics_km = compute_metrics(X_scaled, labels_km)
metrics_km['runtime'] = runtime_km
metrics_km['k'] = selected_k
results['K-Means'] = metrics_km

In [ ]:
# 2. Agglomerative with selected k
print("\n" + "="*60)
print("EXPERIMENT: AGGLOMERATIVE")
print("="*60)
start = time.time()
labels_agg, model_agg = run_agglomerative(X_scaled, k=selected_k, linkage='ward')
runtime_agg = time.time() - start
metrics_agg = compute_metrics(X_scaled, labels_agg)
metrics_agg['runtime'] = runtime_agg
metrics_agg['k'] = selected_k
results['Agglomerative (ward)'] = metrics_agg

In [ ]:
# 3. DBSCAN (tune eps)
# Strategy: Try multiple eps values, select best by silhouette
print("\n" + "="*60)
print("EXPERIMENT: DBSCAN (tuning eps)")
print("="*60)

eps_values = [0.3, 0.5, 0.7, 1.0]
best_dbscan = None
best_dbscan_silhouette = -1

for eps in eps_values:
    labels_db, model_db = run_dbscan(X_scaled, eps=eps, min_samples=50, verbose=False)
    
    # Check if valid clustering (at least 2 clusters, not all noise)
    n_clusters = len(np.unique(labels_db[labels_db >= 0]))
    n_noise = np.sum(labels_db == -1)
    
    if n_clusters >= 2 and n_noise < len(labels_db) * 0.5:  # Less than 50% noise
        metrics_db = compute_metrics(X_scaled, labels_db, verbose=False)
        print(f"  eps={eps}: {n_clusters} clusters, {n_noise} noise ({n_noise/len(labels_db)*100:.1f}%), silhouette={metrics_db['silhouette']:.4f}")
        
        if metrics_db['silhouette'] > best_dbscan_silhouette:
            best_dbscan_silhouette = metrics_db['silhouette']
            best_dbscan = {'eps': eps, 'labels': labels_db, 'metrics': metrics_db, 'model': model_db}
    else:
        print(f"  eps={eps}: INVALID ({n_clusters} clusters, {n_noise/len(labels_db)*100:.1f}% noise)")

if best_dbscan:
    print(f"\n→ Best DBSCAN: eps={best_dbscan['eps']}, silhouette={best_dbscan['metrics']['silhouette']:.4f}")
    labels_db = best_dbscan['labels']
    
    # Time a final run for fair comparison
    start = time.time()
    _, _ = run_dbscan(X_scaled, eps=best_dbscan['eps'], min_samples=50, verbose=False)
    runtime_db = time.time() - start
    
    metrics_db = best_dbscan['metrics']
    metrics_db['runtime'] = runtime_db
    results['DBSCAN'] = metrics_db
else:
    print("\n⚠️  DBSCAN failed to find valid clustering (all eps values resulted in excessive noise or <2 clusters)")
    labels_db = None

---
## E) Algorithm Comparison Table

In [ ]:
# Generate comparison table
comparison_df = plot_cluster_comparison_table(results, save=True)

In [ ]:
# Select champion algorithm
# Priority: Silhouette ≥ 0.35, DB < 1.5, then interpretability, then speed

champion = None
champion_name = None

for algo, metrics in results.items():
    sil = metrics.get('silhouette', 0)
    db = metrics.get('davies_bouldin', 999)
    
    if sil >= 0.35 and db < 1.5:
        if champion is None or sil > champion.get('silhouette', 0):
            champion = metrics
            champion_name = algo

# Fallback: if no algo meets criteria, pick best silhouette
if champion is None:
    best_sil = max(results.items(), key=lambda x: x[1].get('silhouette', -1))
    champion_name = best_sil[0]
    champion = best_sil[1]
    print("⚠️  No algorithm met strict criteria (silhouette ≥ 0.35, DB < 1.5)")
    print(f"   Selecting best by silhouette: {champion_name}\n")

print("="*60)
print("CHAMPION ALGORITHM")
print("="*60)
print(f"  {champion_name}")
print(f"  Silhouette: {champion.get('silhouette', 0):.4f}")
print(f"  Davies-Bouldin: {champion.get('davies_bouldin', 0):.4f}")
print(f"  Calinski-Harabasz: {champion.get('calinski_harabasz', 0):.1f}")
print(f"  Clusters: {champion.get('k', champion.get('n_clusters', '?'))}")
print(f"  Runtime: {champion.get('runtime', 0):.2f}s")
print("="*60 + "\n")

# Set champion labels for interpretation
if champion_name == 'K-Means':
    champion_labels = labels_km
elif champion_name == 'Agglomerative (ward)':
    champion_labels = labels_agg
else:
    champion_labels = labels_db

---
## F) Stability Check (K-Means only)

In [ ]:
# Check K-Means stability across 20 runs
if champion_name == 'K-Means':
    stability_metrics = stability_check(X_scaled, k=selected_k, n_runs=20, verbose=True)
else:
    print("Skipping stability check (champion is not K-Means)")

---
## G) Interpret Champion Clusters

Analyze cluster characteristics and assign interpretations.

In [ ]:
# Describe cluster profiles
feature_cols = ['duration_min', 'distance_km', 'start_hour', 'weekday', 'is_weekend', 'is_member', 'is_round_trip']
profiles = describe_clusters(df_clean, champion_labels, feature_cols=feature_cols, verbose=True)

In [ ]:
# Automatic interpretation
interpretations = interpret_clusters(profiles, verbose=True)

In [ ]:
# Manual refinement (review and adjust interpretations based on domain knowledge)
# You can override automatic interpretations here if needed
print("\n" + "="*60)
print("CLUSTER NAMING (Final)")
print("="*60)
for cluster_id, interp in interpretations.items():
    print(f"Cluster {cluster_id}: {interp}")
print("="*60)

---
## H) Cluster Visualizations

In [ ]:
# 1. Cluster profile heatmap
plot_cluster_profiles(df_clean, champion_labels, feature_cols=feature_cols, save=True)

In [ ]:
# 2. Duration distribution by cluster
plot_cluster_distributions(df_clean, champion_labels, feature='duration_min', save=True)

In [ ]:
# 3. Distance distribution by cluster
plot_cluster_distributions(df_clean, champion_labels, feature='distance_km', save=True)

In [ ]:
# 4. Hour × Weekday heatmaps for each cluster
unique_clusters = np.unique(champion_labels[champion_labels >= 0])  # Exclude noise if present

for cluster_id in unique_clusters:
    plot_hourly_weekday_heatmap(df_clean, champion_labels, cluster_id=cluster_id, save=True)

---
## I) Reflection: What Worked & Policy Implications

### What Worked
✅ **Algorithm Performance**:
- **Champion**: {champion_name} achieved silhouette = {champion['silhouette']:.4f}, DB = {champion['davies_bouldin']:.4f}
- K-Means was fast and interpretable; Agglomerative validated results; DBSCAN struggled with uniform density

✅ **Cluster Interpretability**:
- Clusters align with hypotheses: commuters (weekday peaks), tourists (weekend leisure), last-mile (short trips)
- Clear separation in temporal (start_hour, weekday) and behavioral (duration, is_member) features

✅ **Feature Engineering**:
- 7 features (duration, distance, hour, weekday, is_weekend, is_member, is_round_trip) captured distinct patterns
- Scaling ensured no feature dominated due to scale differences

### Known Biases & Limitations
⚠️ **Seasonal Bias**: Spring/summer data may overrepresent leisure trips; winter validation recommended

⚠️ **Geographic Skew**: Manhattan/Brooklyn dominant; outer boroughs underrepresented

⚠️ **Member Bias**: ~75% members may overshadow casual rider patterns

### Policy Implications (Cluster-Specific)

**For each cluster identified, recommend:**

1. **Weekday Commuters** (if found):
   - Prioritize protected bike lanes on weekday AM/PM corridors
   - Expand stations near office districts and transit hubs
   - Ensure bike availability during peak hours (7-9 AM, 5-7 PM)

2. **Weekend Leisure/Tourists** (if found):
   - Add stations near parks, waterfronts, tourist attractions
   - Design scenic routes (e.g., Brooklyn Bridge, Central Park loops)
   - Market to hotels and visitor centers

3. **Last-Mile Connectors** (if found):
   - Integrate with public transit (bike racks at subway entrances)
   - Ensure high station density near transit hubs
   - Promote "bike + transit" combo passes

4. **Casual/Errand Riders** (if found):
   - Ensure coverage in residential and commercial areas
   - Flexible pricing for mid-duration trips
   - Promote for daily errands (groceries, appointments)

### Unexpected Findings
- [Review cluster profiles and note any surprises: e.g., "reverse commuters" (suburb → city AM, city → suburb PM), "midnight riders" (late-night trips)]
- [Check for clusters that don't fit hypotheses and investigate]

---

## Summary: Capstone 3 Deliverables

✅ **Champion Algorithm**: {champion_name} (k={champion.get('k', champion.get('n_clusters', '?'))})

✅ **Metrics**:
- Silhouette: {champion.get('silhouette', 0):.4f}
- Davies-Bouldin: {champion.get('davies_bouldin', 0):.4f}
- Calinski-Harabasz: {champion.get('calinski_harabasz', 0):.1f}

✅ **Cluster Interpretations**: [List interpretations from above]

✅ **Visualizations** (saved to `reports/figures/`):
- Elbow analysis (4 metrics vs k)
- Cluster profile heatmap
- Duration/distance boxplots by cluster
- Hour × weekday heatmaps per cluster

✅ **Comparison Table**: `reports/cluster_comparison_table.csv`

### Next Steps (Capstone 4)
- Generate 2D PCA projection (mandatory)
- Optional: t-SNE or UMAP for alternative visualization
- Create cluster characteristic summary table
- Spatial analysis (map stations by dominant cluster)
- Update DECISIONS_LOG.md with champion selection rationale

---

*Ready for Capstone 4: Evaluating & Visualizing Clusters* 🚴‍♀️📊🎯